# Register SmartSPIM Data To CCFv3 Mouse Brain Atlas

Atlas reference: http://help.brain-map.org/display/mouseconnectivity/API

In [1]:
import itertools

import itk
import zarr
import ome_zarr
import numpy as np
import itkwidgets

In [2]:
DO_SAVE_TO_DISK = False
DO_VISUALIZE = False

## Load Data

In [3]:
# Download from http://help.brain-map.org/display/mouseconnectivity/API
average_template = itk.imread('data/CCFv3/average_template_10.nrrd', pixel_type=itk.F)

In [4]:
USE_RAW_DATA = False

if USE_RAW_DATA:
    # Convert downloaded zarr data to ITK Image format
    smartspim_zarr = zarr.open('data/Ex_647_Em_690.zarr/4', mode='r')
    arr = np.asarray(smartspim_zarr).astype(np.uint16)
    print(arr.shape)
    smartspim_image = itk.image_from_array(arr[0,0,:,:,:].astype(np.float32))
    smartspim_image.SetSpacing([28.8, 28.98, 32])
else:
    # Get pre-aligned data
    smartspim_image = itk.imread('data/slicer-processed/smartspim-rotated.nrrd', pixel_type=itk.F)
print(type(smartspim_image))

<class 'itk.itkImagePython.itkImageF3'>


In [5]:
if DO_VISUALIZE:
    itkwidgets.view(image)

## Validate Data

We expect the data to have roughly the same extent.

In [6]:
def print_extent(image):
    image_size = itk.size(image)
    for i, j, k in itertools.product([0, image_size[0]],
                                 [0, image_size[1]],
                                 [0, image_size[2]]):
        print(image.TransformIndexToPhysicalPoint([i,j,k]))

fixed_image_size = itk.size(average_template)
print(f'CCF extent:')
print_extent(average_template)
print(f'SmartSPIM extent:')
print_extent(smartspim_image)

CCF extent:
itkPointD3 ([0, 0, 0])
itkPointD3 ([0, 0, 11400])
itkPointD3 ([0, 8000, 0])
itkPointD3 ([0, 8000, 11400])
itkPointD3 ([13200, 0, 0])
itkPointD3 ([13200, 0, 11400])
itkPointD3 ([13200, 8000, 0])
itkPointD3 ([13200, 8000, 11400])
SmartSPIM extent:
itkPointD3 ([0, 0, 0])
itkPointD3 ([5.93392e-12, 8384, -1.34343e-27])
itkPointD3 ([-18547.2, 1.31271e-11, -1.49289e-11])
itkPointD3 ([-18547.2, 8384, -1.49289e-11])
itkPointD3 ([5.53957e-12, 0, -13305.6])
itkPointD3 ([1.14735e-11, 8384, -13305.6])
itkPointD3 ([-18547.2, 1.31271e-11, -13305.6])
itkPointD3 ([-18547.2, 8384, -13305.6])


In [7]:
if DO_VISUALIZE:
    itkwidgets.view(smartspim_image)

## Initialize Registration with `itk`

In [8]:
itk.auto_progress(1)
itk.CenteredTransformInitializer
itk.auto_progress(0)

Loading ITKMesh... Loading ITKMesh... Loading ITKImageFunction... Loading ITKImageFunction... Loading ITKSpatialObjects... Loading ITKSpatialObjects... Loading ITKImageSources... Loading ITKImageSources... Loading ITKImageGrid... Loading ITKImageGrid... Loading ITKFFT... Loading ITKImageCompose... Loading ITKImageCompose... Loading ITKImageStatistics... Loading ITKImageStatistics... Loading ITKPath... Loading ITKPath... Loading ITKImageIntensity... Loading ITKImageIntensity... Loading ITKThresholding... Loading ITKThresholding... Loading ITKConvolution... Loading ITKConvolution... Loading ITKSmoothing... Loading ITKSmoothing... Loading ITKImageGradient... Loading ITKImageGradient... Loading Strain... Loading Strain... Loading SplitComponents... Loading SplitComponents... Loading MeshToPolyData... Loading MeshToPolyData... Loading ITKOptimizers... Loading ITKOptimizers... Loading ITKImageFeature... Loading ITKImageFeature... Loading ITKFiniteDifference... Loading ITKFiniteDifference... 

In [9]:
# Use moments-based initialization to get transform roughly positioning
# sample data on top of CCF data

init_transform = itk.VersorRigid3DTransform[itk.D].New()
init_transform.SetIdentity()

transform_initializer = itk.CenteredTransformInitializer[init_transform, type(average_template), type(smartspim_image)].New()
transform_initializer.SetFixedImage(average_template)
transform_initializer.SetMovingImage(smartspim_image)
transform_initializer.SetTransform(init_transform)
transform_initializer.MomentsOn()

transform_initializer.InitializeTransform()
print(init_transform)

VersorRigid3DTransform (0000026B90AC2CE0)
  RTTI typeinfo:   class itk::VersorRigid3DTransform<double>
  Reference Count: 2
  Modified Time: 1003
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    1 0 0 
    0 1 0 
    0 0 1 
  Offset: [-14285.8, 506.626, -12463.6]
  Center: [7300.82, 3730.63, 5695]
  Translation: [-14285.8, 506.626, -12463.6]
  Inverse: 
    1 0 0 
    0 1 0 
    0 0 1 
  Singular: 0
  Versor: [ 0, 0, 0, 1 ]



In [10]:
# Resample sample data into estimated CCF space
smartspim_image_init = itk.resample_image_filter(smartspim_image, transform=init_transform, use_reference_image=True, reference_image=average_template)

In [11]:
if DO_VISUALIZE:
    itkwidgets.compare(smartspim_image,smartspim_image_init)

In [12]:
if DO_SAVE_TO_DISK:
    itk.imwrite(smartspim_image_init, 'data/output/smartspim_init_moments.mha', compression=True)

## Register with `itk-elastix`

In [13]:
itk.auto_progress(1)
itk.ElastixRegistrationMethod
itk.auto_progress(0)

Loading ITKVoronoi... Loading ITKVoronoi... Loading ITKQuadEdgeMesh... Loading ITKQuadEdgeMesh... Loading ITKIOMeshBase... Loading ITKIOMeshBYU... Loading ITKIOMeshBYU... Loading ITKIOMeshFreeSurfer... Loading ITKIOMeshFreeSurfer... Loading ITKIOMeshGifti... Loading ITKIOMeshGifti... Loading ITKIOMeshOBJ... Loading ITKIOMeshOBJ... Loading ITKIOMeshOFF... Loading ITKIOMeshOFF... Loading ITKIOMeshVTK... Loading ITKIOMeshVTK... Loading ITKIOMeshBase... Loading Elastix... Loading Elastix... 

In [14]:
rigid_parameter_object = itk.ParameterObject.New()
rigid_parameter_object.AddParameterMap(rigid_parameter_object.GetDefaultParameterMap('rigid'))
rigid_parameter_object.AddParameterMap(rigid_parameter_object.GetDefaultParameterMap('affine'))

#print(rigid_parameter_object)

In [15]:
registration_method = itk.ElastixRegistrationMethod[type(average_template), type(smartspim_image)].New(
    fixed_image=average_template,
    moving_image=smartspim_image_init,
    parameter_object=rigid_parameter_object,
    log_to_console=True
)

In [ ]:
# Run registration with `itk-elastix` (will take a few minutes)
registration_method.Update()

In [ ]:
if DO_VISUALIZE:
    itkwidgets.checkerboard(registration_method.GetOutput(), average_template)
    
if DO_WRITE_TO_DISK:    
    itk.imwrite(registration_method.GetOutput(), 'data/output/Ex_647_Em_690_registered.mha', compression=True)
    
    for index in rigid_parameter_object.GetNumberOfParameterMaps():
        registration_method.GetTransformParameterObject().WriteParameterFile(
            registration_method.GetTransformParameterObject().GetParameterMap(index), f'data/output/elastix-transform{index}.h5')